In [3]:
import pandas as pd
kpis = pd.read_csv("../simulation_data/gateway_viability_sweep_ag1_.csv")

In [4]:
kpis

,Unnamed: 0,Net Minting Rate,KPI C,Experiment Name,State Set,Params Set,timestep,param_minimum_stake_servicer,param_minimum_stake_period_servicer,param_minimum_pause_time,...,kpi_a,floating_supply,circulating_supply,dao_value_capture,POKT_burned_cummulative,POKT_minted_cummulative,POKT_net_mint_cummulative,burn_rate_cummulative,mint_rate_cummulative,net_mint_rate_cummulative
0,0,0.001651,0.159556,gateway_viability_sweep_ag1_1,Base,gateway_viability_sweep_ag1_1,365,1.500000e+10,10,10,...,1.0,1.524029e+15,1.520849e+15,0.000193,4.305534e+11,2.942830e+12,2.512277e+12,0.000283,0.001934,0.001651
1,1,0.001692,0.182310,gateway_viability_sweep_ag1_1,Base,gateway_viability_sweep_ag1_1,365,1.500000e+10,10,10,...,1.0,1.524091e+15,1.520525e+15,0.000195,3.926855e+11,2.966376e+12,2.573690e+12,0.000258,0.001950,0.001692
2,2,0.001698,0.164375,gateway_viability_sweep_ag1_2,Base,gateway_viability_sweep_ag1_2,365,1.500000e+10,10,10,...,1.0,1.524101e+15,1.520802e+15,0.000194,3.677418e+11,2.951697e+12,2.583955e+12,0.000242,0.001940,0.001698
3,3,0.001704,0.178243,gateway_viability_sweep_ag1_2,Base,gateway_viability_sweep_ag1_2,365,1.500000e+10,10,10,...,1.0,1.524110e+15,1.520714e+15,0.000194,3.570776e+11,2.950062e+12,2.592984e+12,0.000235,0.001939,0.001704
4,4,0.005313,0.181710,gateway_viability_sweep_ag1_3,Base,gateway_viability_sweep_ag1_3,365,1.500000e+10,10,10,...,1.0,1.529601e+15,1.526338e+15,0.000616,1.189891e+13,9.488008e+13,8.298117e+13,0.000268,0.001932,0.001664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,571,0.002397,0.239130,gateway_viability_sweep_ag1_286,Base,gateway_viability_sweep_ag1_286,365,1.500000e+10,10,10,...,1.0,1.525164e+15,1.524939e+15,0.000277,1.733976e+11,1.218537e+12,1.045140e+12,0.000347,0.002719,0.002372
572,572,0.115944,0.066145,gateway_viability_sweep_ag1_287,Base,gateway_viability_sweep_ag1_287,365,1.500000e+10,10,10,...,1.0,1.697928e+15,1.697097e+15,0.011634,5.000278e+11,3.573416e+12,3.073388e+12,0.009562,0.063204,0.053642
573,573,0.110303,0.067548,gateway_viability_sweep_ag1_287,Base,gateway_viability_sweep_ag1_287,365,1.500000e+10,10,10,...,1.0,1.689345e+15,1.688495e+15,0.011551,4.533601e+11,3.501192e+12,3.047832e+12,0.007555,0.061282,0.053727
574,574,0.107449,0.064157,gateway_viability_sweep_ag1_288,Base,gateway_viability_sweep_ag1_288,365,1.500000e+10,10,10,...,1.0,1.685002e+15,1.684283e+15,0.011135,5.299658e+11,3.544050e+12,3.014084e+12,0.007311,0.059405,0.052094
